In [4]:
# TODO which one?
#git clone https://github.com/lucidrains/iTransformer.git
#import iTransformer
import sys
sys.path.append('/vol/fob-vol7/nebenf21/reinbene/bene/MA/iTransformer') 
from iTransformer import iTransformer
import torch
from pathlib import Path

import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

from utils import data_handling, helpers, training_functions
import config
import pandas as pd

window_size = 96
pred_length = (96, 192, 336, 720)

print("Import succesfull")


Import succesfull


# Select dataset for training

In [5]:
# eu electricity data
data_tensor = data_handling.eu_electricity_to_tensor()
data_dict, standardize_values =  data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to dataloader
dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)

Feature batch shape: torch.Size([32, 96, 208])


948

# Load model that has been trained on electricity dataset

In [5]:
best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.1, 
                   'num_mem_tokens': 4, 'learning_rate': 0.0005}


model_config = {
    'num_variates': data_dict["train"].size(1),
    'lookback_len': window_size,
    'depth': best_parameters["depth"],
    'dim': best_parameters["dim"],
    'num_tokens_per_variate': 1,
    'pred_length': pred_length,
    'dim_head': best_parameters["dim_head"],
    'heads': best_parameters["heads"],
    'attn_dropout': best_parameters["attn_dropout"],
    'ff_mult': best_parameters["ff_mult"],
    'ff_dropout': best_parameters["ff_dropout"],
    'num_mem_tokens': best_parameters["num_mem_tokens"],
    'use_reversible_instance_norm': False,
    'reversible_instance_norm_affine': False,
    'flash_attn': True
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_path = config.CONFIG_MODEL_LOCATION["base"] / "electricity_base_epoch_15_loss_0.15809133648872375.pt"

#model_path = "/vol/fob-vol7/nebenf21/reinbene/bene/MA/outputs/models/revin/trial_epoch_15_loss_0.12157157808542252.pt"
#model_path = "/vol/fob-vol7/nebenf21/reinbene/bene/MA/outputs/models/stationary/stationary_for_transfer.pt"
model_path = "/vol/fob-vol7/nebenf21/reinbene/bene/MA/outputs/models/base/electricity_base_epoch_15_loss_0.15809133648872375.pt"
checkpoint = torch.load(model_path)
model = iTransformer(**model_config).to(device)
model.load_state_dict(checkpoint['model_state_dict'])


# defining all needed instances
optimizer = optim.Adam(model.parameters(), lr=best_parameters["learning_rate"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH["revin"])

Using device: cuda


In [11]:
checkpoint["model_state_dict"]["reversible_instance_norm.beta"] = checkpoint["model_state_dict"]["reversible_instance_norm.beta"][:67, :]
checkpoint["model_state_dict"]["reversible_instance_norm.gamma"] = checkpoint["model_state_dict"]["reversible_instance_norm.gamma"][:67, :]

In [9]:
checkpoint["model_state_dict"]["reversible_instance_norm.beta"].size()

torch.Size([67, 1])

# Load bavaria dataset and try zero shot prediciton on the test data

In [2]:
# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
data_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# create a smaller subset of the train dataset
#percentage = 0.1
#ten_percent_length = int(percentage * data_dict["train"].shape[0])
#data_dict["train"] = data_dict["train"][:ten_percent_length, :]

# convert to datalaoder
dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)

Feature batch shape: torch.Size([32, 96, 67])


In [15]:
# first we try zero shot

metrics = helpers.full_eval(model, dataloader_test, device)
metrics

for eval_metric, value in metrics[96].items():
    metrics[96][eval_metric] = value.item()

metrics_df = pd.DataFrame.from_dict(metrics[96], orient='index')
metrics_df.rename(columns={0: "metrics"}, inplace=True)

metrics_df.to_csv(f"{config.CONFIG_OUTPUT_PATH['iTransformer_baseline']}/transfer.csv")

Epoch: Validating: 100%|██████████| 71/71 [00:01<00:00, 49.79it/s]


In [16]:
metrics

{96: {'mse': 0.19589269161224365,
  'mae': 0.414349764585495,
  'p10': 0.22389757633209229,
  'p50': 0.4057007431983948,
  'p90': 0.6108494400978088},
 192: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0},
 336: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0},
 720: {'mse': 0, 'mae': 0, 'p10': 0, 'p50': 0, 'p90': 0}}

# Fine-tune pretrained model on small subset of training dataset and predict on the same test dataset

In [11]:
# now we train on a small subset of the training data for multiple epochs and evaluate again

epoch = 10
training_functions.train_one_epoch(epoch, model, device, dataloader_train, dataloader_validation, optimizer, scheduler, writer)

metrics = training_functions.full_eval(model, dataloader_test)
metrics

for eval_metric, value in metrics[96].items():
    metrics[96][eval_metric] = value.item()

metrics_df = pd.DataFrame.from_dict(metrics[96], orient='index')
metrics_df.rename(columns={0: key}, inplace=True)

metrics_df.to_csv(f"{config.CONFIG_OUTPUT_PATH['iTransformer_baseline']}/metrics_{key}_epochs{epoch}_revin.csv")


NameError: name 'train_one_epoch' is not defined

In [ ]:
# we load the pretrained model again and do full fine tuning on the target dataset

best_parameters = {'depth': 2, 'dim': 256, 'dim_head': 56, 'heads': 4, 'attn_dropout': 0.2, 'ff_mult': 4, 'ff_dropout': 0.1, 
                   'num_mem_tokens': 4, 'learning_rate': 0.0005}


model_config = {
    'num_variates': data_dict["train"].size(1),
    'lookback_len': window_size,
    'depth': best_parameters["depth"],
    'dim': best_parameters["dim"],
    'num_tokens_per_variate': 1,
    'pred_length': pred_length,
    'dim_head': best_parameters["dim_head"],
    'heads': best_parameters["heads"],
    'attn_dropout': best_parameters["attn_dropout"],
    'ff_mult': best_parameters["ff_mult"],
    'ff_dropout': best_parameters["ff_dropout"],
    'num_mem_tokens': best_parameters["num_mem_tokens"],
    'use_reversible_instance_norm': False,
    'reversible_instance_norm_affine': False,
    'flash_attn': True
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_path = config.CONFIG_MODEL_LOCATION[normalization_strategie]
checkpoint = torch.load(model_path)
model = iTransformer(**model_config).to(device)
model.load_state_dict(checkpoint['model_state_dict'])


# defining all needed instances
optimizer = optim.Adam(model.parameters(), lr=best_parameters["learning_rate"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
writer = SummaryWriter(log_dir=config.CONFIG_LOGS_PATH[normalization_strategie])

# Fine-tuning with the full bavaria dataset and prediciton on the same test dataset

In [ ]:
# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
data_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)

# convert to datalaoder
dataloader_train, dataloader_validation, dataloader_test = data_handling.convert_data(data_dict, window_size, pred_length)

In [ ]:
# now we train on a small subset of the training data for multiple epochs and evaluate again

epoch = 10
training_functions.train_one_epoch(epoch, model, device, dataloader_train, dataloader_validation, optimizer, scheduler, writer)

metrics = training_functions.full_eval(model, dataloader_test)
metrics

for eval_metric, value in metrics[96].items():
    metrics[96][eval_metric] = value.item()

metrics_df = pd.DataFrame.from_dict(metrics[96], orient='index')
metrics_df.rename(columns={0: key}, inplace=True)

metrics_df.to_csv(f"{config.CONFIG_OUTPUT_PATH['iTransformer_baseline']}/metrics_{key}_epochs{epoch}_revin.csv")
